# CDAE - Movie Recommendation Top-k

## Library

In [1]:
import os
import sys
import random
import time

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import numpy as np
import pandas as pd
from scipy import sparse

## Config

In [2]:
import yaml

def load_config(config_file):
    with open(config_file, 'r') as stream:
        try:
            config = yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            print(exc)
    return config

cfg = load_config('config.yaml')

In [3]:
if torch.cuda.is_available():
    print('CUDA is available')
    cfg['device'] = True

device = torch.device('cuda' if cfg['device'] else 'cpu')
device

CUDA is available


device(type='cuda')

In [4]:
random.seed(cfg['seed'])
np.random.seed(cfg['seed'])
torch.manual_seed(cfg['seed'])


In [5]:
raw_data = pd.read_csv(os.path.join(cfg['DATA_DIR'], cfg['data']), header=0, usecols=[0, 1])

In [6]:
raw_data['user'].nunique()

from sklearn.model_selection import train_test_split


# train, test = train_test_split(raw_data, test_size=0.2, random_state=cfg['seed'])

In [10]:
from typing import Iterable, Tuple

import numpy as np
import pandas as pd
import torch.utils.data as data
from sklearn.model_selection import train_test_split


class CDAEData(data.Dataset):
    def __init__(self, data: np.ndarray) -> None:
        super(CDAEData, self).__init__()

        self.data = data

    def __len__(self) -> int:
        return len(self.data)

    def __getitem__(self, index) -> None:
        return index, self.data[index]

def get_count(data: np.ndarray, id: str) -> pd.DataFrame:
    r"""Helper function to get the count of each id.

    Parameters
    ----------
    data : np.ndarray
        Data which consists of user_id, item_id.

    id : str
        The name of the id.

    Returns
    -------
    pd.DataFrame
        The count of each id.
    """
    count_groupby = data[[id]].groupby(id, as_index=False).size()

    return count_groupby


def preprocess(data: pd.DataFrame) -> Tuple[np.ndarray, np.ndarray, int, int]:
    r"""Helper function to preprocess data.

    Parameters
    ----------
    data : pd.DataFrame
        Data which consists of user_id, item_id.

    Returns
    -------
    Tuple[np.ndarray, np.ndarray, int, int]
    """
    num_users = data["user"].nunique()
    num_items = data["item"].nunique()

    user_activity = get_count(data=data, id="user")
    item_popularity = get_count(data=data, id="item")

    # Shuffle User Indices
    unique_uid = user_activity['user'].unique()
    unique_sid = item_popularity['item'].unique()

    # Indexing
    user2id = dict((uid, i) for (i, uid) in enumerate(unique_uid))
    item2id = dict((sid, i) for (i, sid) in enumerate(unique_sid))

    # # np.random.seed(seed)
    # idx_perm = np.random.permutation(unique_uid.size)
    # unique_uid = unique_uid[idx_perm]

    # # Split Train / Test User
    # tr_users = unique_uid[:-3000]
    # te_users = unique_uid[-3000:]

    # train = data.loc[raw_data['user'].isin(tr_users)]
    # test = data.loc[raw_data['user'].isin(te_users)]

    # Create a directory to save preprocessed data
    pro_dir = os.path.join('pro_sg')

    if not os.path.exists(pro_dir):
        os.makedirs(pro_dir)

    with open(os.path.join(pro_dir, 'unique_sid.txt'), 'w') as f:
        for sid in unique_sid:
            f.write('%s\n' % sid)

    with open(os.path.join(pro_dir, 'unique_uid.txt'), 'w') as f:
        for uid in unique_uid:
            f.write('%s\n' % uid)

    train = numerize(data, user2id, item2id)
    # train = numerize(train, user2id, item2id)
    # test = numerize(test, user2id, item2id)

    train_mat = _to_matrix(data=train, num_users=num_users, num_items=num_items)
    test_mat = _to_matrix(data=train, num_users=num_users, num_items=num_items)

    return train_mat, test_mat, num_users, num_items

def numerize(
        tp: pd.DataFrame,
        user2id: dict,
        item2id: dict,
        ) -> pd.DataFrame:
    r"""Helper function to numerize user and item ids.

    Parameters
    ----------
    tp : pd.DataFrame
        Data which consists of user_id, item_id.

    user2id : dict
        A dictionary which maps user_id to user index.

    item2id : dict
        A dictionary which maps item_id to item index.

    Returns
    -------
    pd.DataFrame
        Numerized data.
    """
    uid = tp['user'].apply(lambda x: user2id[x])
    sid = tp['item'].apply(lambda x: item2id[x])
    return pd.DataFrame(data={'uid': uid, 'sid': sid}, columns=['uid', 'sid'])


def _to_matrix(
    data: Iterable,
    num_users: int = None,
    num_items: int = None,
) -> np.ndarray:
    r"""Helper function to convert an iterable object into the form of matrix.

    Parameters
    ----------
    data : Iterable
        Movie Lens prepocessed data which consists of user, item.
    num_users : int, optional
        The number of users, by default None
    num_items : int, optional
        The number of items, by default None

    Returns
    -------
    np.ndarray
        Rating matrix
    """
    if isinstance(data, np.ndarray):
        data = np.array(data, dtype=int)

    # initialize a matrix
    mat = np.zeros((num_users, num_items))

    for user, item in data[['uid', 'sid']].values:
        mat[user, item] = 1

    return mat

In [11]:
train_mat, test_mat, num_users, num_items = preprocess(raw_data)

In [12]:
train_set = CDAEData(train_mat)
test_set = CDAEData(test_mat)

In [13]:
train_loader = data.DataLoader(train_set, batch_size=cfg['batch_size'], shuffle=True)
test_loader = data.DataLoader(test_set, batch_size=cfg['batch_size'], shuffle=False)

In [14]:
class CDAE(nn.Module):
    r"""Collaborative Denoising Auto-Encoder

    Parameters
    ----------
    num_users : int
        _description_
    num_items : int
        _description_
    num_hidden_units : int
        _description_
    corruption_ratio : float
        _description_

    References
    ----------
    [1] Wu, Yao, et al. "Collaborative denoising auto-encoders for top-n
        recommender systems." Proceedings of the ninth ACM international
        conference on web search and data mining. 2016.

    """

    def __init__(
        self,
        num_users: int,
        num_items: int,
        num_hidden_units: int,
        corruption_ratio: float,
    ) -> None:
        super(CDAE, self).__init__()

        self.num_users = num_users
        self.num_items = num_items
        self.num_hidden_units = num_hidden_units
        self.corruption_ratio = corruption_ratio

        # CDAE consists of user embedding, encoder, decoder
        self.user_embedding = nn.Embedding(num_users, num_hidden_units)
        self.encoder = nn.Linear(num_items, num_hidden_units)
        self.decoder = nn.Linear(num_hidden_units, num_items)

        # Set to use GPU
        self.cuda()

    def forward(
        self, user_idx: torch.Tensor, matrix: torch.Tensor
    ) -> torch.Tensor:
        # Apply corruption
        matrix = F.dropout(
            matrix, p=self.corruption_ratio, training=self.training
        )
        encoder = torch.tanh(
            self.encoder(matrix) + self.user_embedding(user_idx)
        )
        return self.decoder(encoder)

    def train_one_epoch(
        self, train_loader: data.DataLoader, optimizer: optim.Optimizer
    ) -> torch.Tensor:
        r"""Train a single epoch.

        Parameters
        ----------
        data_loader : data.DataLoader
            Training data loader
        optimizer : optim.Optimizer
            An optimizer

        Returns
        -------
        torch.Tensor
            Loss value after training one epoch.
        """
        loss = 0
        # Turn training mode on
        self.train()

        # By using BCEWithLogitsLoss, the return value of 'forward()' method
        # is not using sigmoid function.
        loss_f = CDAELoss(model=self, lambda_reg=0.01)

        for (indices, input_mat) in train_loader:
            indices = indices.cuda()
            input_mat = input_mat.float().cuda()
            self.zero_grad()

            predict_mat = self.forward(user_idx=indices, matrix=input_mat)
            batch_loss = loss_f(input=predict_mat, target=input_mat)
            batch_loss.backward()
            optimizer.step()
            loss += batch_loss

        return loss / len(train_loader)

    def predict(self, train_loader: data.DataLoader) -> np.ndarray:
        r"""Predict items per users.
        Observations that already seen by each user are masked with `-inf`.

        Parameters
        ----------
        train_loader : data.DataLoader
            Training data loader

        Returns
        -------
        np.ndarray
            Prediction matrix which of dimension is same as training data.
        """
        with torch.no_grad():
            preds = np.zeros_like(train_loader.dataset.data)

            for (indices, input_mat) in train_loader:
                indices = indices.cuda()
                input_mat = input_mat.float().cuda()
                batch_pred = torch.sigmoid(self.forward(indices, input_mat))
                batch_pred = batch_pred.masked_fill(
                    input_mat.bool(), float("-inf")
                )

                indices = indices.detach().cpu().numpy()
                preds[indices] = batch_pred.detach().cpu().numpy()

        return preds

class CDAELoss(nn.Module):
    def __init__(self, model: nn.Module, lambda_reg=0.01):
        """
        Initialize the CDAE loss module with L2 regularization.

        Parameters:
        - model: The CDAE model instance to apply regularization.
        - lambda_reg: Regularization strength.
        """
        super(CDAELoss, self).__init__()
        self.model = model
        self.lambda_reg = lambda_reg
        self.bce_with_logits_loss = nn.BCEWithLogitsLoss()

    def forward(self, input, target):
        """
        Compute the CDAE loss as the sum of BCE loss and L2 regularization.

        Parameters:
        - input: The predicted values from the CDAE.
        - target: The true values (actual user-item interactions).

        Returns:
        - The computed loss value.
        """
        # Compute the binary cross-entropy loss
        bce_loss = self.bce_with_logits_loss(input, target)

        # Compute the L2 regularization term
        l2_reg = torch.tensor(0.).cuda()
        for param in self.model.parameters():
            l2_reg += torch.norm(param)**2
        l2_reg_loss = self.lambda_reg * l2_reg

        # Total loss
        total_loss = bce_loss + l2_reg_loss
        return total_loss


In [15]:
model = CDAE(
    num_users=num_users,
    num_items=num_items,
    num_hidden_units=cfg['num_hidden_units'],
    corruption_ratio=cfg['corruption_ratio'],
).to(device)

In [16]:
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [17]:
from tqdm import tqdm

for epoch in tqdm(range(cfg['n_epochs'])):
        loss = model.train_one_epoch(train_loader, optimizer)
        print(f"[Epoch {epoch}]:: Loss: {loss}")

  1%|▏         | 1/80 [00:02<03:18,  2.52s/it]

[Epoch 0]:: Loss: 113995.3515625


  2%|▎         | 2/80 [00:04<02:38,  2.03s/it]

[Epoch 1]:: Loss: 93897.765625


  4%|▍         | 3/80 [00:05<02:26,  1.90s/it]

...

[Epoch 77]:: Loss: 0.6819154024124146


 99%|█████████▉| 79/80 [02:22<00:01,  1.80s/it]

[Epoch 78]:: Loss: 0.6819136738777161


100%|██████████| 80/80 [02:23<00:00,  1.80s/it]

[Epoch 79]:: Loss: 0.6818909645080566


In [18]:
preds = model.predict(train_loader=train_loader)

In [19]:
def recall_at_k(actual: np.ndarray, pred: np.ndarray, top_k: int) -> float:
    r"""_summary_

    Parameters
    ----------
    actual : np.ndarray
        _description_
    pred : np.ndarray
        _description_
    top_k : int
        _description_

    Returns
    -------
    float
        _description_
    """
    actual = np.asarray(actual).astype(np.bool)

    # Get indices of the top_k predicted scores
    top_k_pred_indices = np.argsort(pred)[-top_k:]

    # Create a binary array of the same shape as actual, where only the top k predictions are True
    top_k_preds = np.zeros_like(pred, dtype=np.bool)
    top_k_preds[top_k_pred_indices] = True

    # Calculate the number of relevant items retrieved in the top k predictions
    relevant_and_retrieved = np.sum(actual & top_k_preds)

    # Calculate the total number of relevant items
    total_relevant = np.sum(actual)

    # Handle the case where there are no relevant items
    if total_relevant == 0:
        return 0.0

    # Calculate recall
    recall = relevant_and_retrieved / total_relevant

    return recall

def map_at_k(actual: np.ndarray, pred: np.ndarray, top_k: int) -> float:
    r"""Mean average precision at k.

    Parameters
    ----------
    actual : np.ndarray
        A matrix with actual values.
    pred : np.ndarray
        A matrix with predictions.
    top_k : int

    Returns
    -------
    float
        Mean average precision at k
    """
    if not _assert_same_dimension(actual, pred):
        raise AssertionError("Two input matrices should have same dimension.")

    map_ = 0

    num_users = len(pred)
    top_k_items = _topk(input=pred, k=top_k)

    for i in range(num_users):
        actual_item = set(actual[i].nonzero()[0])
        pred_item = top_k_items[i]

        map_ += _ap_at_k(actual=actual_item, pred=pred_item, top_k=top_k)

    return map_ / num_users

def _ap_at_k(actual: np.array, pred: np.array, top_k: int) -> float:
    r"""Avearge precision at k

    Parameters
    ----------
    actual : np.array
        A list of item are to be predicted
    pred : np.array
        A list of predicted items
    top_k : int

    Returns
    -------
    float
        Average precision at k
    """

    if len(pred) > top_k:
        pred = pred[:top_k]

    p, cnt = 0, 0

    if not actual:
        return 0.0

    for idx, item in enumerate(pred):
        if item in actual:
            cnt += 1
            p += cnt / (idx + 1)

    return 0.0 if cnt == 0 else p / min(top_k, len(actual))

def _topk(input: np.ndarray, k: int) -> np.ndarray:
    r"""Returns indices of k largest element of the given input matrix along
    the horizontal axis.

    Parameters
    ----------
    input : np.ndarray
        _description_
    k : int
        _description_

    Returns
    -------
    np.ndarray
        _description_
    """
    return np.argsort(input)[:, -k:][:, ::-1]

def _assert_same_dimension(actual: np.ndarray, pred: np.ndarray) -> bool:
    r"""Check the actual matrix and the prediction have same dimension.

    Parameters
    ----------
    actual : np.ndarray
        Actual values
    pred : np.ndarray
        Predicted values

    Returns
    -------
    bool
    """
    return actual.shape == pred.shape


In [20]:
eval_result = map_at_k(actual=test_mat, pred=preds, top_k=10)

print(f"MAP@10: {eval_result:.6f}")

MAP@10: 0.000000


In [21]:
eval_result = recall_at_k(actual=test_mat, pred=preds, top_k=10)

print(f"RECALL@10: {eval_result:.6f}")

/tmp/ipykernel_420536/2323554715.py:18: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  actual = np.asarray(actual).astype(np.bool)
/tmp/ipykernel_420536/2323554715.py:24: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  top_k_preds = np.zeros_like(pred, dtype=np.bool)


RECALL@10: 0.219061


In [22]:
watchedm=raw_data.groupby('user')['item'].apply(list)

In [23]:
watchedm

user
11        [4643, 170, 531, 616, 2140, 2722, 2313, 2688, ...
14        [8961, 1396, 471, 2105, 1042, 1947, 1269, 2394...
18        [1952, 1283, 3507, 4280, 51084, 593, 318, 356,...
25        [261, 22, 2161, 3255, 372, 1093, 428, 175, 214...
31        [260, 1196, 1210, 7153, 4993, 5952, 1270, 5855...
                                ...                        
138473    [524, 3354, 1025, 6565, 69757, 2085, 32, 55282...
138475    [1639, 1673, 1148, 246, 2019, 1267, 1172, 1235...
138486    [2694, 1994, 2723, 441, 2288, 637, 2013, 2423,...
138492    [2115, 908, 58, 2700, 2599, 1500, 1358, 1288, ...
138493    [3174, 2872, 48780, 2662, 2840, 1566, 2857, 20...
Name: item, Length: 31360, dtype: object

In [24]:
unique_sid = pd.read_csv(os.path.join('pro_sg/unique_sid.txt'),sep=" ",header=None)
unique_uid = pd.read_csv(os.path.join('pro_sg/unique_uid.txt'),sep=" ",header=None)

In [25]:
id2show = dict((i, sid) for (i, sid) in enumerate(unique_sid.squeeze()))
id2profile = dict((i, pid) for (i, pid) in enumerate(unique_uid.squeeze()))

In [26]:
temp = pd.DataFrame(preds)

column=list(temp.columns)
origin_mid=[id2show[x] for x in column]

row=list(temp.index)
origin_uid=[id2profile[x] for x in row]

temp.columns=origin_mid
temp.index=origin_uid

In [27]:
from tqdm import tqdm
sumbission=dict()
sumbission={'user': [],'item': []}
sumbission

for row in tqdm(temp.iterrows(),total=31360):
    userid=row[0]
    movies=row[1]
    watchedmovies=watchedm.get(userid, [])

    for _ in range(10):
        sumbission['user'].append(userid)

    itemp=[]
    for movie in reversed(list(movies.sort_values().index)):
        if len(itemp)==10:
            break
        else:
            if movie not in watchedmovies:
                itemp.append(movie)

    sumbission['item']+=itemp

sumbission=pd.DataFrame(sumbission)
sumbission.sort_values('user', inplace=True)
sumbission.to_csv('submission.csv', index=False)

100%|██████████| 31360/31360 [01:02<00:00, 499.18it/s]
